# Introduction



***Final Project (`Segmify`)***

Team Name : 3PecintaRoni    
Members :     
Adhy Arya Hendrata    
Daffa Darwin    
Vicky Eldora Wuisan      

object : Creating a machine learning-based recommendation system aims to provide      
        customers with the best selection of goods based on their needs.

# ***Import Libraries & Packages***

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
from feature_engine.outliers import Winsorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import re

# ***Data Loading***

In [2]:
# Load the CSV file
df_ori = pd.read_csv('data.csv', encoding='ISO-8859-1')

# duplicate dataset
df = df_ori.copy()

# Display the first 3 rows to understand the structure of the data
df.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom


In [3]:
df.tail(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,12/9/2011 12:50,4.95,12680.0,France


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [5]:
df.duplicated().sum()

5268

In [6]:
df.nunique()

InvoiceNo      25900
StockCode       4070
Description     4223
Quantity         722
InvoiceDate    23260
UnitPrice       1630
CustomerID      4372
Country           38
dtype: int64

In [7]:
# mengecek missing value
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [9]:
# menampilkan kolom yang hanya berisi integer dan float
num_col_1 = df.select_dtypes(include=np.number).columns.tolist()

# menampilkan kolom yang hanya berisi object
cat_col_1 = df.select_dtypes(include=['object']).columns.tolist()

print(f'numerical columns: {num_col_1}')
print(f'categorical columns: {cat_col_1}')

df_num = df[num_col_1]
df_cat = df[cat_col_1]

numerical columns: ['Quantity', 'UnitPrice', 'CustomerID']
categorical columns: ['InvoiceNo', 'StockCode', 'Description', 'InvoiceDate', 'Country']


In [10]:
df.head(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom


In [11]:
df.nunique()

InvoiceNo      25900
StockCode       4070
Description     4223
Quantity         722
InvoiceDate    23260
UnitPrice       1630
CustomerID      4372
Country           38
dtype: int64

# ***Feature Engginering***

## ***Handling Outlier***

In [12]:
def clean_data(df):
    # Remove duplicate rows
    df.drop_duplicates(inplace=True)

    # Replace "ID" to "Id" in column names
    df.columns = df.columns.str.replace('ID', 'Id')
    
    # Replace uppercase letters in the middle of words with underscores
    df.columns = df.columns.str.replace(r'(?<!^)(?=[A-Z])', '_', regex=True).str.lower()
    
    # Reset index
    df.reset_index(drop=True, inplace=True)

    # Clean string columns
    for col in df.select_dtypes(include=['object']).columns:
        # Remove leading and trailing spaces
        df[col] = df[col].str.strip()

        # Remove special characters and convert to lowercase
        df[col] = df[col].str.replace(r'[\*\?!]', '', regex=True).str.lower()
    
    return df

def identify_distribution(df):
    normal, skew, extreme_skew = [], [], []

    # Identify numerical columns
    df_num = df.select_dtypes(include=['float64', 'int64'])

    # Determine the skewness of each numerical column
    for col in df_num.columns:
        skewness = df[col].skew()
        if -0.5 < skewness < 0.5:
            normal.append(col)
        elif -1 <= skewness <= -0.5 or 0.5 <= skewness <= 1:
            skew.append(col)
        elif skewness < -1 or skewness > 1:
            extreme_skew.append(col)
    return normal, skew, extreme_skew

def handle_missing_value(df):
    # Calculate the percentage of missing values for each column
    missing_percent = df.isnull().mean() * 100

    # Fill or drop missing values based on percentage
    for col in df.columns:
        if missing_percent[col] > 5:
            if df[col].dtype == 'object':  # Categorical data
                df[col].fillna(df[col].mode()[0], inplace=True)
            else:  # Numerical data
                skewness = df[col].dropna().skew()
                if abs(skewness) < 0.5:  # Normally distributed
                    df[col].fillna(df[col].mean(), inplace=True)
                else:  # Skewed distribution
                    df[col].fillna(df[col].median(), inplace=True)
        else:
            df.dropna(subset=[col], inplace=True)

    return df

def handle_outliers(df):
    normal, skew, extreme_skew = identify_distribution(df)

    # Handle extreme skewed data
    if extreme_skew:
        winsoriser_extreme_skew = Winsorizer(capping_method='iqr', tail='both', fold=3, variables=extreme_skew)
        df = winsoriser_extreme_skew.fit_transform(df)
    
    # Handle skewed data
    if skew:
        winsoriser_skew = Winsorizer(capping_method='iqr', tail='both', fold=1.5, variables=skew)
        df = winsoriser_skew.fit_transform(df)
    
    # Handle normally distributed data
    if normal:
        winsoriser_normal = Winsorizer(capping_method='gaussian', tail='both', fold=3, variables=normal)
        df = winsoriser_normal.fit_transform(df)
    
    # Debugging statements to ensure Winsorizer is applied
    for col in normal + skew + extreme_skew:
        if col in df.columns:
            print(f"Outliers handled for column: {col}")

    return df

def find_skewed_boundaries(df, variable, distance):
    IQR = df[variable].quantile(0.75) - df[variable].quantile(0.25)
    lower_boundary = df[variable].quantile(0.25) - (IQR * distance)
    upper_boundary = df[variable].quantile(0.75) + (IQR * distance)
    return upper_boundary, lower_boundary

def calculate_outliers_percentage(df, variable, distance):    
    upper_boundary, lower_boundary = find_skewed_boundaries(df, variable, distance)
    outliers = df[(df[variable] < lower_boundary) | (df[variable] > upper_boundary)]
    per = (len(outliers) / len(df)) * 100
    print('Percentage of outliers in {}: {:.2f}%'.format(variable, per))

def diagnostic_plots(df, variable):
    plt.figure(figsize=(16, 4))

    # Histogram
    plt.subplot(1, 2, 1)
    sns.histplot(df[variable], bins=30)
    plt.title(f'Histogram of {variable}')

    # Boxplot
    plt.subplot(1, 2, 2)
    sns.boxplot(y=df[variable])
    plt.title(f'Boxplot of {variable}')

    plt.show()

# Main data processing
df = clean_data(df)
df = handle_missing_value(df)
df_clean = handle_outliers(df)

normal, skew, extreme_skew = identify_distribution(df)

# Display percentage of outliers before and after outlier handling
print("Outliers percentage before handling:")
for col in normal:
    calculate_outliers_percentage(df, col, 1.5)
for col in skew:
    calculate_outliers_percentage(df, col, 1.5)
for col in extreme_skew:
    calculate_outliers_percentage(df, col, 3)

print("\nOutliers percentage after handling:")
for col in normal:
    calculate_outliers_percentage(df_clean, col, 1.5)
for col in skew:
    calculate_outliers_percentage(df_clean, col, 1.5)
for col in extreme_skew:
    calculate_outliers_percentage(df_clean, col, 3)

# Generate diagnostic plots before and after trimming for all variables
print("\nDiagnostic plots before and after trimming:")
for col in normal + skew + extreme_skew:
    print(f'\n{col} - Before Trimming')
    diagnostic_plots(df, col)
    print(f'\n{col} - After Trimming')
    diagnostic_plots(df_clean, col)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_19544\1340654869.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)


,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
0,536365,85123a,white hanging heart t-light holder,6.0,12/1/2010 8:26,2.55,17517.801354,united kingdom
1,536365,71053,white metal lantern,6.0,12/1/2010 8:26,3.39,17517.801354,united kingdom
2,536365,84406b,cream cupid hearts coat hanger,8.0,12/1/2010 8:26,2.75,17517.801354,united kingdom
3,536365,84029g,knitted union flag hot water bottle,6.0,12/1/2010 8:26,3.39,17517.801354,united kingdom
4,536365,84029e,red woolly hottie white heart.,6.0,12/1/2010 8:26,3.39,17517.801354,united kingdom


In [ ]:
df_clean.head()

In [13]:
df_clean['invoice_date'] = pd.to_datetime(df_clean['invoice_date'])
df_clean[['customer_id','quantity']] = df_clean[['customer_id','quantity']].astype(int)

In [14]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535122 entries, 0 to 535121
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   invoice_no    535122 non-null  object        
 1   stock_code    535122 non-null  object        
 2   description   535122 non-null  object        
 3   quantity      535122 non-null  int32         
 4   invoice_date  535122 non-null  datetime64[ns]
 5   unit_price    535122 non-null  float64       
 6   customer_id   535122 non-null  int32         
 7   country       535122 non-null  object        
dtypes: datetime64[ns](1), float64(1), int32(2), object(4)
memory usage: 28.6+ MB


In [15]:
df_clean.isnull().sum()

invoice_no      0
stock_code      0
description     0
quantity        0
invoice_date    0
unit_price      0
customer_id     0
country         0
dtype: int64

In [ ]:
df.duplicated().sum()

In [16]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535122 entries, 0 to 535121
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   invoice_no    535122 non-null  object        
 1   stock_code    535122 non-null  object        
 2   description   535122 non-null  object        
 3   quantity      535122 non-null  int32         
 4   invoice_date  535122 non-null  datetime64[ns]
 5   unit_price    535122 non-null  float64       
 6   customer_id   535122 non-null  int32         
 7   country       535122 non-null  object        
dtypes: datetime64[ns](1), float64(1), int32(2), object(4)
memory usage: 28.6+ MB


In [17]:
# melihat berapa unique value
df_clean.nunique()

invoice_no      24446
stock_code       3848
description      4177
quantity           73
invoice_date    22309
unit_price        486
customer_id      3263
country            38
dtype: int64

In [18]:
# menampilkan nama kolom
df_clean.columns

Index(['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date',
       'unit_price', 'customer_id', 'country'],
      dtype='object')

# ***Recommended System Methods***

In [19]:
# Gabungkan baris berdasarkan 'Description'
df_grouped = df_clean.groupby('description').agg({
    'stock_code': 'first',
    'quantity': 'sum',
    'invoice_date': 'first',  # atau bisa gunakan min/max tergantung kebutuhan
    'unit_price': 'mean',  # atau bisa gunakan sum/mean tergantung kebutuhan
    'customer_id': 'first',  # atau bisa gunakan mode/min/max tergantung kebutuhan
    'country': 'first'
}).reset_index()

In [20]:
# Update the function to handle non-string values
def categorize_description(description):
    if not isinstance(description, str):
        return 'Miscellaneous'
    
    categories = []
    
    # Define the category keywords
    category_keywords = {
        'Candles': ['candles'],
        'Decorations': ['decoration', 'ornament', 'garland','holder','flannel','platter','balloons'],
        'Kitchenware': ['mug', 'plate', 'bowl', 'jampot','cutlery','jars','tea','bottle','container','teacup','tissues','napkins','baking'],
        'Stationery': ['notebook', 'pen', 'pencil', 'paper','sticker','pen'],
        'Toys': ['toy', 'game', 'puzzle','block','children','dolly'],
        'Furniture': ['chair', 'table', 'sofa', 'stool'],
        'Clothing': ['shirt', 'dress', 'trousers', 'sock','woolly'],
        'Electronics': ['lamp', 'light', 'clock'],
        'Accessories': ['bag', 'scarf', 'belt','charm','rucksack','backpack','earrings'],
        'Miscellaneous': ['misc', 'various']
    }
    
    # Lowercase description for case insensitive matching
    description_lower = description.lower()
    
    # Check keywords and assign categories
    for category, keywords in category_keywords.items():
        if any(re.search(r'\b' + keyword + r'\b', description_lower) for keyword in keywords):
            categories.append(category)
        if len(categories) == 2:  # Maximum of 2 categories
            break
    
    if not categories:
        categories.append('Miscellaneous')
    
    return ' '.join(categories)

# Apply the function to the Description column to create the Category column
df_grouped['category'] = df_grouped['description'].apply(categorize_description)

# Display the first few rows with the new Category column
df_grouped.head()

,description,stock_code,quantity,invoice_date,unit_price,customer_id,country,category
0,,21275,-1047,2010-12-03 16:50:00,0.000000,15287,united kingdom,Miscellaneous
1,10 colour spaceboy pen,22418,5393,2010-12-01 11:41:00,1.050917,14307,united kingdom,Stationery
2,12 coloured party balloons,22436,2030,2010-12-02 13:32:00,0.703952,17517,united kingdom,Miscellaneous
3,12 daisy pegs in wood box,21448,344,2010-12-01 11:49:00,1.916667,17517,united kingdom,Miscellaneous
4,12 egg house painted wood,22282,148,2010-12-06 09:58:00,12.739200,15287,united kingdom,Miscellaneous


In [21]:
df_grouped.tail()

,description,stock_code,quantity,invoice_date,unit_price,customer_id,country,category
4172,zinc t-light holder stars small,23144,3867,2011-04-21 09:36:00,0.942446,16133,united kingdom,Decorations Electronics
4173,zinc top 2 door wooden shelf,21275,8,2010-12-08 11:33:00,12.770000,14952,united kingdom,Miscellaneous
4174,zinc willie winkie candle stick,84832,2624,2010-12-01 09:41:00,1.085971,15311,united kingdom,Miscellaneous
4175,zinc wire kitchen organiser,23143,30,2011-05-04 13:17:00,8.425625,17517,united kingdom,Miscellaneous
4176,zinc wire sweetheart letter tray,23137,83,2011-04-21 16:53:00,3.976522,16750,united kingdom,Miscellaneous


In [22]:
df_grouped = df_grouped[df_grouped['unit_price'] != 0.000000]

In [23]:
# Display a sample of the data to see different descriptions and their categories
df_grouped.sample(10)

,description,stock_code,quantity,invoice_date,unit_price,customer_id,country,category
1547,gold cosmetics bag with butterfly,20860,1,2010-12-13 16:46:00,2.100000,15287,united kingdom,Accessories
3768,turquoise christmas tree,35591t,5,2010-12-01 11:57:00,1.250000,13057,united kingdom,Miscellaneous
2490,parisienne sewing box,23111,288,2011-05-11 13:20:00,12.332968,14277,france,Miscellaneous
1742,herb marker rosemary,22917,1683,2010-12-01 12:43:00,0.850331,14729,united kingdom,Miscellaneous
447,blue easter egg hunt start post,21461,8,2011-01-16 16:23:00,2.495000,17517,united kingdom,Miscellaneous
4147,yellow/pink ceramic candle holder,37491a,6,2010-12-01 17:06:00,3.307500,15287,united kingdom,Decorations
554,box of 6 mini vintage crackers,23318,3074,2011-07-04 10:17:00,2.732905,15287,united kingdom,Miscellaneous
3289,set of 6 ice cream skittles,84715,16,2011-01-30 14:05:00,2.950000,13148,united kingdom,Miscellaneous
4098,wrap flower shop,23547,1571,2011-07-20 14:39:00,0.420000,17511,united kingdom,Miscellaneous
3464,slate tile natural hanging,22185,1410,2010-12-01 13:21:00,1.949207,17517,united kingdom,Miscellaneous


In [24]:
df_grouped['category'].unique()

array(['Stationery', 'Miscellaneous', 'Kitchenware', 'Toys', 'Candles',
       'Decorations', 'Decorations Stationery', 'Accessories',
       'Stationery Accessories', 'Electronics', 'Clothing',
       'Decorations Electronics', 'Decorations Clothing', 'Furniture',
       'Kitchenware Electronics', 'Furniture Electronics',
       'Kitchenware Accessories', 'Candles Electronics',
       'Toys Accessories', 'Kitchenware Toys', 'Decorations Toys',
       'Stationery Electronics', 'Toys Clothing',
       'Kitchenware Stationery', 'Candles Kitchenware',
       'Stationery Furniture', 'Stationery Toys', 'Decorations Furniture',
       'Kitchenware Furniture'], dtype=object)

In [25]:
df_grouped.nunique()

description     4031
stock_code      3802
quantity        1760
invoice_date    1395
unit_price      3442
customer_id      654
country           20
category          29
dtype: int64

In [26]:
Category = ' '
for g in df_grouped['category']:
  Category += g+' '

Category = list(set(Category.split(' ')))[1:]

Category

['Accessories',
 'Stationery',
 'Electronics',
 'Candles',
 'Kitchenware',
 'Miscellaneous',
 'Furniture',
 'Decorations',
 'Clothing',
 'Toys']

In [27]:
gen_desc = [[] for i in range(len(Category))]

for dat in df_grouped['category']:
  for i,g in enumerate(Category):
    if g in dat.split(' '):
      gen_desc[i].append(1)
    else:
      gen_desc[i].append(0)

In [28]:
gen_mv_dat = pd.DataFrame(np.array(gen_desc).T,columns=Category)
gen_mv_dat

,Accessories,Stationery,Electronics,Candles,Kitchenware,Miscellaneous,Furniture,Decorations,Clothing,Toys
0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
4026,0,0,1,0,0,0,0,1,0,0
4027,0,0,0,0,0,1,0,0,0,0
4028,0,0,0,0,0,1,0,0,0,0
4029,0,0,0,0,0,1,0,0,0,0


In [29]:
title_df_grouped = df_grouped[['description']].reset_index(drop=True)
df_grouped_vector = pd.concat([title_df_grouped,gen_mv_dat],axis=1)
df_grouped_vector.set_index('description',inplace=True)
df_grouped_vector


,Accessories,Stationery,Electronics,Candles,Kitchenware,Miscellaneous,Furniture,Decorations,Clothing,Toys
description,,,,,,,,,,
10 colour spaceboy pen,0,1,0,0,0,0,0,0,0,0
12 coloured party balloons,0,0,0,0,0,1,0,0,0,0
12 daisy pegs in wood box,0,0,0,0,0,1,0,0,0,0
12 egg house painted wood,0,0,0,0,0,1,0,0,0,0
12 hanging eggs hand painted,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
zinc t-light holder stars small,0,0,1,0,0,0,0,1,0,0
zinc top 2 door wooden shelf,0,0,0,0,0,1,0,0,0,0
zinc willie winkie candle stick,0,0,0,0,0,1,0,0,0,0


In [30]:
def cosine_sim(vect1,vect2):
  norm_1 = np.linalg.norm(vect1)
  norm_2 = np.linalg.norm(vect2)

  cos_sim = (vect1 @ vect2) / (norm_1 * norm_2)
  return cos_sim

In [31]:
df_grouped[df_grouped['description']=='white hanging heart t-light holder'][['description','category']]

,description,category
3980,white hanging heart t-light holder,Decorations Electronics


In [32]:
df_grouped[df_grouped['description']=='white metal lantern'][['description','category']]

,description,category
3988,white metal lantern,Miscellaneous


In [33]:
cosine_sim(df_grouped_vector.loc['white hanging heart t-light holder'], df_grouped_vector.loc['white metal lantern'])

0.0

In [34]:
def recsys(df_grouped, top_N):
  cossim = pd.Series([cosine_sim(df_grouped_vector.loc[df_grouped],x) for x in df_grouped_vector.values],index=df_grouped_vector.index).drop(index=df_grouped)
  print(f'You like {df_grouped}, so based on our recommender system, We recommend you to see:')
  for i,mv in enumerate(cossim.sort_values(ascending=False)[:top_N].index):
    print(f'{i+1}. {mv}')

In [35]:
recsys('white hanging heart t-light holder',3)

You like white hanging heart t-light holder, so based on our recommender system, We recommend you to see:
1. mirror love bird t-light holder
2. heart trellistriple t-light holder
3. heart t-light holder willie winkie
